In [1]:
import numpy as np
from datasets import load_all_data_mnist, load_all_data_cifar10, load_all_data_fashion_mnist, load_all_data_apply_vgg_cifar10
from utils import create_batch_data
import modules
import activations
import layers
from criterions import RealMSELoss

ModuleNotFoundError: No module named 'utils'

In [2]:
BATCH_SIZE = 256
EPOCH = 1
LR = 0.01
PRINT = 10
FLATTEN = False
# 0, MNIST; 1, FashionMNIST; 2, CIFAR10; 3, VGG-CIFAR10
DATASET_MODE = 2

In [3]:
# data fetching
load_path = '../../data'
if DATASET_MODE == 0:
    train_data, train_label, test_data, test_label = load_all_data_mnist(load_path, flatten=FLATTEN)
elif DATASET_MODE == 1:
    train_data, train_label, test_data, test_label = load_all_data_fashion_mnist(load_path, flatten=FLATTEN)
elif DATASET_MODE == 2:
    train_data, train_label, test_data, test_label = load_all_data_cifar10(load_path, flatten=FLATTEN)
elif DATASET_MODE == 3:
    train_data, train_label, test_data, test_label = load_all_data_apply_vgg_cifar10(load_path, flatten=FLATTEN)
else:
    train_data, train_label, test_data, test_label = None, None, None, None
train_data, train_label, test_data, test_label = create_batch_data(train_data, train_label, test_data, test_label, BATCH_SIZE)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
model_arr = [
    layers.RealPiNetSecondOrderConvLayer(3, 64, (4, 4), stride=(2, 2), padding=(1, 1, 1, 1)),
    layers.RealPiNetSecondOrderConvLayer(64, 256, (4, 4), stride=(2, 2), padding=(1, 1, 1, 1)),
    layers.GAPTruncation(),
    layers.RealLinearLayer(256, 10)
]

model = modules.Network(model_arr)
criterion = RealMSELoss()

In [5]:
for epoch in range(EPOCH):
    tot_loss = 0
    for train_idx, (train_data_batch, train_label_batch) in enumerate(zip(train_data, train_label)):
        # train
        preds = model.forward(train_data_batch)

        tot_loss += criterion.forward(preds, train_label_batch)
        propagated_error = criterion.error_derivative()

        model.backprop(propagated_error)
        model.optimize(LR)

        if train_idx == 0 or (train_idx + 1) % PRINT == 0:
            tot_acc = 0
            tot_sample = 0
            for train_acc_idx, (test_data_batch, test_label_batch) in enumerate(zip(test_data, test_label)):
                # train accuracy
                preds = model.forward(test_data_batch)
                pred_args = np.argmax(preds, axis=1)

                tot_acc += np.count_nonzero(pred_args == test_label_batch)
                tot_sample += test_data_batch.shape[0]
            accuracy = tot_acc / tot_sample
            if train_idx != 0:
                tot_loss = tot_loss / PRINT
            print('epoch: {}, idx: {}, accuracy: {}, loss: {}'.format(epoch + 1, train_idx + 1, accuracy, tot_loss))
            tot_loss = 0

epoch: 1, idx: 1, accuracy: 0.0895, loss: 1.0078885631559544
epoch: 1, idx: 10, accuracy: 0.0903, loss: 0.8995184890630504
epoch: 1, idx: 20, accuracy: 0.102, loss: 0.9847955383652061
epoch: 1, idx: 30, accuracy: 0.1013, loss: 0.9633101828112511
epoch: 1, idx: 40, accuracy: 0.1036, loss: 0.9388229922630075
epoch: 1, idx: 50, accuracy: 0.1288, loss: 0.9154936129841307
epoch: 1, idx: 60, accuracy: 0.1684, loss: 0.9022156871460949
epoch: 1, idx: 70, accuracy: 0.192, loss: 0.8958614811725724
epoch: 1, idx: 80, accuracy: 0.2061, loss: 0.8885328246043894
epoch: 1, idx: 90, accuracy: 0.2122, loss: 0.8821011846561426
epoch: 1, idx: 100, accuracy: 0.2351, loss: 0.8776488053694468
epoch: 1, idx: 110, accuracy: 0.23, loss: 0.8765227317219217
epoch: 1, idx: 120, accuracy: 0.2529, loss: 0.8715747493119428
epoch: 1, idx: 130, accuracy: 0.2569, loss: 0.8719109400641895
epoch: 1, idx: 140, accuracy: 0.263, loss: 0.8649194609111781
epoch: 1, idx: 150, accuracy: 0.2726, loss: 0.8615075119855495
epoch: 1